In [ ]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.python.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
base_sure_directory = '/content/drive/MyDrive/base_sure_enrish'

In [ ]:
import os
import numpy as np
import tensorflow as tf
import random
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input

def extract_label_from_filename(filename):
    # Exemple : on extrait un label à partir du nom du fichier
    # À adapter selon votre logique
    return int(filename.split('-')[1].split('_')[0])

class ImageLabelGenerator(Sequence):
    def __init__(self, image_dir, image_filenames, batch_size=32, target_size=(300, 300), shuffle=True, augment=False):
        self.image_dir = image_dir
        self.image_filenames = image_filenames
        self.batch_size = batch_size
        self.target_size = target_size
        self.shuffle = shuffle
        self.augment = augment
        self.on_epoch_end()

    def __len__(self):
        # Retourne le nombre de lots par epoch
        return int(np.floor(len(self.image_filenames) / self.batch_size))

    def __getitem__(self, index):
        batch_filenames = self.image_filenames[index * self.batch_size:(index + 1) * self.batch_size]
        images = []
        labels = []

        for filename in batch_filenames:
            label = extract_label_from_filename(filename)
            img_path = os.path.join(self.image_dir, filename)
            image = load_img(img_path, target_size=self.target_size)
            image = img_to_array(image)

            # Appliquer la data augmentation si activée
            if self.augment:
                image = self.random_transform(image)

            # Appliquer le prétraitement attendu par le modèle pré-entraîné
            image = preprocess_input(image)
            images.append(image)
            labels.append(label)

        return np.array(images), np.array(labels)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.image_filenames)

    def random_transform(self, image):
        # Convertir l'image en tenseur pour appliquer les transformations
        image_tensor = tf.convert_to_tensor(image, dtype=tf.float32)

        # Flip horizontal aléatoire
        if random.random() < 0.5:
            image_tensor = tf.image.flip_left_right(image_tensor)

        # Variation de luminosité aléatoire
        image_tensor = tf.image.random_brightness(image_tensor, max_delta=0.1)

        # Variation de contraste aléatoire
        image_tensor = tf.image.random_contrast(image_tensor, lower=0.8, upper=1.2)

        # Rotation aléatoire entre -20° et +20° via la couche RandomRotation
        rotation_layer = tf.keras.layers.RandomRotation(factor=20/360, fill_mode='reflect')
        image_tensor = rotation_layer(tf.expand_dims(image_tensor, axis=0))[0]

        # Zoom aléatoire :
        # On choisit un facteur entre 0.8 et 1.0, on recadre puis redimensionne à la taille originale
        scale = random.uniform(0.8, 1.0)
        h, w, c = image_tensor.shape
        new_h = tf.cast(scale * h, tf.int32)
        new_w = tf.cast(scale * w, tf.int32)
        image_tensor = tf.image.random_crop(image_tensor, size=[new_h, new_w, c])
        image_tensor = tf.image.resize(image_tensor, [h, w])

        return image_tensor.numpy()




In [ ]:
all_images = [f for f in os.listdir(base_sure_directory) if f.endswith('.jpg')]

train_files, val_files = train_test_split(all_images, test_size=0.2, random_state=42, shuffle=True)

train_generator = ImageLabelGenerator(base_sure_directory, train_files, batch_size=32, augment=True)
val_generator = ImageLabelGenerator(base_sure_directory, val_files, batch_size=32, shuffle=False, augment=False)

In [ ]:
import os
import numpy as np
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# =========================
# 1) Paramètres & Générateurs
# =========================
IMG_HEIGHT = 300
IMG_WIDTH = 300
NUM_CLASSES = 9
BATCH_SIZE = 32

# Supposons que train_generator et val_generator soient déjà définis
# et qu'ils appliquent un prétraitement compatible ResNet (par ex. tf.keras.applications.resnet50.preprocess_input).
# Sinon, assurez-vous de l'ajouter dans votre code de génération.

# =========================
# 2) Construction du modèle
# =========================
# Charger ResNet50 pré-entraîné sur ImageNet, sans la tête fully-connected
base_model = ResNet50(
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
    include_top=False,
    weights='imagenet'
)

# Phase 1 : geler toutes les couches du ResNet
for layer in base_model.layers:
    layer.trainable = False

# Construction de la tête de classification
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)  # Dropout modéré
x = layers.Dense(
    128,
    activation='relu',
    kernel_regularizer=regularizers.l2(1e-5)
)(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = models.Model(inputs=base_model.input, outputs=outputs)

# =========================
# 3) Phase 1 : Entraîner la tête
# =========================
# On choisit un LR plutôt modéré (ex: 1e-4)
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

checkpoint_phase1 = ModelCheckpoint(
    "best_phase1.h5",
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)

history_phase1 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,  # Ajustez selon la taille de votre dataset
    callbacks=[checkpoint_phase1, early_stop, reduce_lr]
)

# Recharger les meilleurs poids de la Phase 1
model.load_weights("best_phase1.h5")

# =========================
# 4) Phase 2 : Fine-tuning
# =========================
# Débloquer, par exemple, les 50 dernières couches du ResNet50
for layer in base_model.layers[-50:]:
    layer.trainable = True

# Recompiler avec un LR plus faible pour affiner les couches dégélées
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

checkpoint_phase2 = ModelCheckpoint(
    "best_phase2.h5",
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)
early_stop_2 = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr_2 = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)

history_phase2 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=80,  # Ajustez selon vos besoins
    callbacks=[checkpoint_phase2, early_stop_2, reduce_lr_2]
)

# Recharger le meilleur modèle obtenu en Phase 2 (optionnel)
model.load_weights("best_phase2.h5")

# =========================
# 5) Évaluation finale
# =========================
loss, accuracy = model.evaluate(val_generator)
print("Validation accuracy finale :", accuracy)


In [ ]:
# =========================
# 6) Affichage des courbes
# =========================
# 1. Récupérer et concaténer les métriques Phase 1 + Phase 2
train_loss = history_phase1.history['loss'] + history_phase2.history['loss']
val_loss   = history_phase1.history['val_loss'] + history_phase2.history['val_loss']

train_acc  = history_phase1.history['accuracy'] + history_phase2.history['accuracy']
val_acc    = history_phase1.history['val_accuracy'] + history_phase2.history['val_accuracy']

# 2. Tracer la Loss (Train + Val)
plt.figure(figsize=(10, 6))
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Courbe de Loss (Phase 1 + Phase 2)')
plt.legend()
plt.show()

# 3. Tracer l’Accuracy (Train + Val)
plt.figure(figsize=(10, 6))
plt.plot(train_acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Courbe d’Accuracy (Phase 1 + Phase 2)')
plt.legend()
plt.show()